<a href="https://colab.research.google.com/github/elichen/aoc2023/blob/main/Day_23_A_Long_Walk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
input = """#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#""".split('\n')

In [ ]:
input = [x.rstrip() for x in open("input.txt").readlines()]

In [ ]:
from collections import deque

def find_paths(map):
    rows, cols = len(map), len(map[0])
    directions = {'^': (-1, 0), '>': (0, 1), 'v': (1, 0), '<': (0, -1)}

    def is_valid(x, y, prev_dir):
        if 0 <= x < rows and 0 <= y < cols:
            if map[x][y] == '.' or map[x][y] in directions:
                return prev_dir is None or map[x][y] == '.' or directions[map[x][y]] == prev_dir
        return False

    def bfs():
        queue = deque([(0, i, {(0, i)}) for i in range(cols) if map[0][i] == '.'])
        paths = []

        while queue:
            x, y, visited = queue.popleft()
            if x == rows - 1:
                paths.append(visited)
                continue

            for dx, dy in directions.values():
                nx, ny = x + dx, y + dy
                if is_valid(nx, ny, (dx, dy)) and (nx, ny) not in visited:
                    new_visited = set(visited)
                    new_visited.add((nx, ny))
                    queue.append((nx, ny, new_visited))

        return paths

    return bfs()

# paths = find_paths(input)
# max([len(x)-1 for x in paths])

In [ ]:
def plot_path_on_map(map_data, path):
    # Copy the original map
    new_map = [list(row) for row in map_data]

    # Mark the path on the new map
    for x, y in path:
        new_map[x][y] = 'X'

    # Convert each row back to a string
    new_map = [''.join(row) for row in new_map]
    print("---")
    print('\n'.join(new_map))

In [ ]:
from functools import lru_cache

def longest_path_set(grid):
    rows, cols = len(grid), len(grid[0])
    start = (0, grid[0].index('.'))
    end_row = rows - 1

    # Convert the grid into a more manageable format
    grid = [list(row) for row in grid]

    @lru_cache(maxsize=10**4)
    def dfs(node, visited):
        visited = set(visited)

        if node[0] == end_row:
            return len(visited)  # Reached the end row

        x, y = node
        visited.add(node)
        while True:
            neighbors = [(x + dx, y + dy) for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]]
            valid_neighbors = [(nx, ny) for nx, ny in neighbors if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] != '#' and (nx, ny) not in visited]

            if len(valid_neighbors) == 1:
                # Only one path, continue along this path
                node = valid_neighbors[0]
                visited.add(node)
                x, y = node
            else:
                break

        if node[0] == end_row:
            return len(visited)
        if len(valid_neighbors) == 0:
            return 0

        # Multiple paths, explore each and choose the one with the longest path
        longest_len = 0
        for neighbor in valid_neighbors:
            current_path = dfs(neighbor, frozenset(visited))
            if current_path > longest_len:
                longest_len = current_path

        return longest_len

    return dfs(start, frozenset())

# Test the function with the provided grid
longest_path_set(input)

155

In [ ]:
from collections import deque

def parse_map(map_str):
    # Parse the map string into a 2D grid
    return [list(row) for row in map_str]

def is_path(cell):
    # Determine if a cell is a path (not a wall)
    return cell != '#'

def get_neighbors(x, y, grid):
    # Get valid neighboring cells (N, S, E, W)
    neighbors = []
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and is_path(grid[nx][ny]):
            neighbors.append((nx, ny))
    return neighbors

def bfs_explore_corrected(map_str):
    grid = parse_map(map_str)
    rows, cols = len(grid), len(grid[0])

    # Find the starting and ending points
    start = [(0, y) for y in range(cols) if is_path(grid[0][y])][0]
    end = [(rows - 1, y) for y in range(cols) if is_path(grid[rows - 1][y])][0]

    # BFS exploration
    queue = deque([(start,start,0,set([start]))])
    graph = {start:{}}

    while queue:
        (x,y),prev,dist,visited = queue.popleft()

        neighbors = get_neighbors(x, y, grid)
        if len(neighbors) > 2 or (x,y) == end:
            if (x, y) not in graph: graph[(x, y)] = {}
            graph[prev][(x,y)] = dist
            graph[(x,y)][prev] = dist
            visited = set([(x,y)])
            prev = (x,y)
            dist = 0

        for nx, ny in neighbors:
            if (nx, ny) != prev and (nx, ny) not in visited and ((nx,ny) not in graph or prev not in graph[(nx,ny)]):
                queue.append(((nx, ny), prev, dist+1, visited | {(nx,ny)}))

    return start, end, graph

# Test the corrected function
start, end, graph = bfs_explore_corrected(input)

In [ ]:
def dfs_find_longest_path_length_corrected(graph, current_node, end_node, visited, current_distance):
    # Mark the current node as visited
    visited.add(current_node)

    # If we reach the end node, return the current distance
    if current_node == end_node:
        visited.remove(current_node)  # Backtrack before returning
        return current_distance

    # Initialize the longest path length as the current distance
    longest_path_length = current_distance

    # Explore neighbors
    for neighbor, distance in graph.get(current_node, {}).items():
        if neighbor not in visited:
            path_length = dfs_find_longest_path_length_corrected(graph, neighbor, end_node, visited.copy(), current_distance + distance)
            longest_path_length = max(longest_path_length, path_length)

    # Backtrack
    visited.remove(current_node)

    return longest_path_length

# Find the longest path length with corrected function
dfs_find_longest_path_length_corrected(graph, start, end, set(), 0)

6534